In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
import random
from sklearn.preprocessing import MinMaxScaler
import sys
import time
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

from IPython.display import HTML
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# DATA LOADING
data=pd.read_csv("../input/malware-detection-tejas/train.csv") # Train Data
identifier=pd.read_csv("../input/malware-detection-tejas/test.csv") 
data1=pd.read_csv("../input/malware-detection-tejas/test.csv") # Test Data
has=data['HasDetections']

In [ ]:
# Columns managed by Ayush Mishra and Aarushi Goenka

columns=['EngineVersion',
 'ProductName',
 'AppVersion',
 'AvSigVersion',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'Platform',
 'Processor',
 'OsVer',
 'OsBuild',
 'OsSuite',
 'OsPlatformSubRelease',
 'OsBuildLab',
 'SkuEdition',
 'IsProtected',
 'IeVerIdentifier',
 'SmartScreen',
 'Firewall',
 'Census_MDC2FormFactor',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_InternalBatteryNumberOfCharges']

data=data[columns]
data1=data1[columns]

In [ ]:
# Filling of NULL VALUES and other DATA CLEANING

data.IsProtected.fillna(0.0,inplace=True)
data.IeVerIdentifier.fillna(137.0,inplace=True)
data.SmartScreen.fillna('UNK',inplace=True)
data.Firewall.fillna(1.0,inplace=True)
data.Census_OEMNameIdentifier.fillna(2668.0,inplace=True)
data.Census_OEMModelIdentifier.fillna(313586.0,inplace=True)
data.Census_ProcessorCoreCount.fillna(4.0,inplace=True)
data.Census_ProcessorManufacturerIdentifier.fillna(5.0,inplace=True)
data.Census_ProcessorModelIdentifier.fillna(1998.0,inplace=True)
data.Census_PrimaryDiskTotalCapacity.fillna(476940.0,inplace=True)
data.Census_PrimaryDiskTypeName.fillna('HDD',inplace=True)
data.Census_SystemVolumeTotalCapacity.fillna(28542.0,inplace=True)
data.Census_TotalPhysicalRAM.fillna(4096.0,inplace=True)
data.Census_ChassisTypeName.fillna('Notebook',inplace=True)
data.Census_InternalPrimaryDiagonalDisplaySizeInInches.fillna(15.5,inplace=True)
data.Census_InternalPrimaryDisplayResolutionHorizontal.fillna(1366.0,inplace=True)
data.Census_InternalPrimaryDisplayResolutionVertical.fillna(768.0,inplace=True)
data.Census_InternalBatteryNumberOfCharges.fillna(0.0,inplace=True)
data.RtpStateBitfield.fillna(7.0,inplace=True)
data.AVProductStatesIdentifier.fillna(-1,inplace=True)
data.AVProductsInstalled.fillna(-1,inplace=True)
data.AVProductsEnabled.fillna(-1,inplace=True)
data.OrganizationIdentifier.fillna(0,inplace=True)
data.GeoNameIdentifier.fillna(0,inplace=True)

#GROUPING CityIdentifier with CountryIdentifier and filling its null values
data['CityIdentifier']=data[['CountryIdentifier','CityIdentifier']].groupby(['CountryIdentifier']).transform(lambda x: x.fillna(x.mode()[0])).CityIdentifier



#REPLACE FUNCTIONS

data.SmartScreen.replace(to_replace='requireadmin',value='RequireAdmin',inplace=True)
data.SmartScreen.replace(to_replace='off',value='Off',inplace=True)
data.SmartScreen.replace(to_replace='on',value='On',inplace=True)
data.SmartScreen.replace(to_replace='0',value='Off',inplace=True)
data.SmartScreen.replace(to_replace='&#x02;',value='RequireAdmin',inplace=True)
data.SmartScreen.replace(to_replace='&#x01;',value='RequireAdmin',inplace=True)

data.Census_MDC2FormFactor.replace(to_replace='SmallTablet',value='Tablet',inplace=True)
data.Census_MDC2FormFactor.replace(to_replace='LargeTablet',value='Tablet',inplace=True)
data.Census_MDC2FormFactor.replace(to_replace='SmallServer',value='Server',inplace=True)
data.Census_MDC2FormFactor.replace(to_replace='MediumServer',value='Server',inplace=True)
data.Census_MDC2FormFactor.replace(to_replace='LargeServer',value='Server',inplace=True)

data.Census_ProcessorManufacturerIdentifier.replace(to_replace=10.0,value=5.0,inplace=True)
data.Census_ProcessorManufacturerIdentifier.replace(to_replace=3.0,value=5.0,inplace=True)
data.Census_ProcessorManufacturerIdentifier.replace(to_replace=4.0,value=5.0,inplace=True)
data.Census_ChassisTypeName.replace(to_replace='Unknown',value='UNKNOWN',inplace=True)

data.ProductName.replace(to_replace='mseprerelease',value='mse',inplace=True)
data.ProductName.replace(to_replace='windowsintune',value='win8defender',inplace=True)
data.ProductName.replace(to_replace='scep',value='win8defender',inplace=True)



# SAME REPEATED FOR TEST DATA

data1.IsProtected.fillna(0.0,inplace=True)
data1.IeVerIdentifier.fillna(137.0,inplace=True)
data1.SmartScreen.fillna('UNK',inplace=True)
data1.Firewall.fillna(1.0,inplace=True)
data1.Census_OEMNameIdentifier.fillna(2668.0,inplace=True)
data1.Census_OEMModelIdentifier.fillna(313586.0,inplace=True)
data1.Census_ProcessorCoreCount.fillna(4.0,inplace=True)
data1.Census_ProcessorManufacturerIdentifier.fillna(5.0,inplace=True)
data1.Census_ProcessorModelIdentifier.fillna(1998.0,inplace=True)
data1.Census_PrimaryDiskTotalCapacity.fillna(476940.0,inplace=True)
data1.Census_PrimaryDiskTypeName.fillna('HDD',inplace=True)
data1.Census_SystemVolumeTotalCapacity.fillna(28542.0,inplace=True)
data1.Census_TotalPhysicalRAM.fillna(4096.0,inplace=True)
data1.Census_ChassisTypeName.fillna('Notebook',inplace=True)
data1.Census_InternalPrimaryDiagonalDisplaySizeInInches.fillna(15.5,inplace=True)
data1.Census_InternalPrimaryDisplayResolutionHorizontal.fillna(1366.0,inplace=True)
data1.Census_InternalPrimaryDisplayResolutionVertical.fillna(768.0,inplace=True)
data1.Census_InternalBatteryNumberOfCharges.fillna(0.0,inplace=True)
data1.OsBuildLab.fillna('17134.1.amd64fre.rs4_release.180410-1804',inplace=True)
data1.Census_PowerPlatformRoleName.fillna('Mobile',inplace=True)
data1.RtpStateBitfield.fillna(7.0,inplace=True)
data1.AVProductStatesIdentifier.fillna(-1,inplace=True)
data1.AVProductsInstalled.fillna(-1,inplace=True)
data1.AVProductsEnabled.fillna(-1,inplace=True)
data1.OrganizationIdentifier.fillna(0,inplace=True)
data1.GeoNameIdentifier.fillna(0,inplace=True)

data1['CityIdentifier']=data1[['CountryIdentifier','CityIdentifier']].groupby(['CountryIdentifier']).transform(lambda x: x.fillna(x.mode()[0])).CityIdentifier



# REPLACE FUNCTIONS
data1.SmartScreen.replace(to_replace='requireadmin',value='RequireAdmin',inplace=True)
data1.SmartScreen.replace(to_replace='off',value='Off',inplace=True)
data1.SmartScreen.replace(to_replace='on',value='On',inplace=True)
data1.SmartScreen.replace(to_replace='0',value='Off',inplace=True)
data1.SmartScreen.replace(to_replace='&#x02;',value='RequireAdmin',inplace=True)
data1.SmartScreen.replace(to_replace='&#x01;',value='RequireAdmin',inplace=True)

data1.Census_MDC2FormFactor.replace(to_replace='SmallTablet',value='Tablet',inplace=True)
data1.Census_MDC2FormFactor.replace(to_replace='LargeTablet',value='Tablet',inplace=True)
data1.Census_MDC2FormFactor.replace(to_replace='SmallServer',value='Server',inplace=True)
data1.Census_MDC2FormFactor.replace(to_replace='MediumServer',value='Server',inplace=True)
data1.Census_MDC2FormFactor.replace(to_replace='LargeServer',value='Server',inplace=True)

data1.Census_ProcessorManufacturerIdentifier.replace(to_replace=10.0,value=5.0,inplace=True)
data1.Census_ProcessorManufacturerIdentifier.replace(to_replace=3.0,value=5.0,inplace=True)
data1.Census_ProcessorManufacturerIdentifier.replace(to_replace=4.0,value=5.0,inplace=True)
data1.Census_ChassisTypeName.replace(to_replace='Unknown',value='UNKNOWN',inplace=True)

data1.ProductName.replace(to_replace='mseprerelease',value='mse',inplace=True)
data1.ProductName.replace(to_replace='windowsintune',value='win8defender',inplace=True)
data1.ProductName.replace(to_replace='scep',value='win8defender',inplace=True)
data1.ProductName.replace(to_replace='fep',value='win8defender',inplace=True)





# GROUPING OF CATEGORIES SO THAT NUMBER OF CATEGORIES REDUCE


version_id={137.0 : 137.0,
            117.0 : 117.0,
            108.0 : 108.0,
            111.0 : 111.0,
            98.0 : 98.0,
           135.0 : 135.0,
           74.0 : 74.0,
           53.0 : 53.0,
           94.0 : 94.0,
           105.0 : 105.0,
           333.0 : 333.0,
           107.0 : 107.0,
           103.0 : 103.0,
           96.0 : 96.0,
           76.0 : 76.0,
           114.0 : 114.0,
           323.0 : 323.0,
           71.0 : 71.0,
            41.0 : 41.0,
           335.0 : 335.0}

arr=list(data.IeVerIdentifier.unique())
for val in arr:
    if(val not in version_id.keys()):
        if(val>=250):
            version_id[val]=333.0
        elif (val>=100 and val<250):
            version_id[val]=137.0
        elif (val>=50 and val<100):
            version_id[val]=98.0
        elif (val<50):
            version_id[val]=53.0
data.IeVerIdentifier = data.IeVerIdentifier.apply(lambda x: version_id.get(x,x))
data1.IeVerIdentifier = data1.IeVerIdentifier.apply(lambda x: version_id.get(x,x))



proc_count={4.0 : 4.0,
            2.0 : 2.0,
            8.0 : 8.0,
            1.0 : 2.0,
            12.0 : 12.0,
            6.0 : 4.0,
            3.0 : 2.0}
arr=list(data.Census_ProcessorCoreCount.unique())
arr
for val in arr:
    if val not in proc_count:
        if(val>=10):
            proc_count[val]=16.0
        else:
            proc_count[val]=8.0
data.Census_ProcessorCoreCount = data.Census_ProcessorCoreCount.apply(lambda x: proc_count.get(x,x))
data1.Census_ProcessorCoreCount = data1.Census_ProcessorCoreCount.apply(lambda x: proc_count.get(x,x))


primary_disk={476940.0 : 476940.0,
              953869.0 : 953869.0,
              305245.0 : 305245.0,
              122104.0 : 122104.0,
             244198.0 : 244198.0,
              29820.0 : 29820.0,
              238475.0 : 238475.0,
              114473.0 : 114473.0,
              715404.0 : 715404.0,
              228936.0 : 228936.0,
              152627.0 : 152627.0,
              59640.0 : 59640.0,
              29824.0 : 29824.0,
              488386.0 : 488386.0,}
arr=list(data.Census_PrimaryDiskTotalCapacity.unique())
for val in arr:
    if val not in primary_disk.keys():
        if(val>=900000.0):
            primary_disk[val]=953869.0
        elif(val>=400000.0 and val<900000):
            primary_disk[val]=476940
        elif(val>=200000.0):
            primary_disk[val]=305245
        elif(val>=100000.0):
            primary_disk[val]=122104.0
        elif (val>=50000):
            primary_disk[val]=59640.0
        else:
            primary_disk[val]=29824.0
data.Census_PrimaryDiskTotalCapacity = data.Census_PrimaryDiskTotalCapacity.apply(lambda x: primary_disk.get(x,x))
data1.Census_PrimaryDiskTotalCapacity = data1.Census_PrimaryDiskTotalCapacity.apply(lambda x: primary_disk.get(x,x))


phy_ram={4096.0 : 4096.0,
         8192.0 : 8192.0,
         2048.0 : 2048.0,
         16384.0 : 16384.0,
         6144.0 : 6144.0,
         3072.0 : 3072.0,
         12288.0: 12288.0,
         1024.0 : 1024.0,
         32768.0 : 32768.0}
arr=list(data.Census_TotalPhysicalRAM.unique())
for val in arr:
    if val not in phy_ram.keys():
        if(val>=8192.0):
            phy_ram[val]=8192.0
        elif (val>=2000.0):
            phy_ram[val]=4096.0
        else:
            phy_ram[val]=2048.0
data.Census_TotalPhysicalRAM = data.Census_TotalPhysicalRAM.apply(lambda x: phy_ram.get(x,x))
data1.Census_TotalPhysicalRAM = data1.Census_TotalPhysicalRAM.apply(lambda x: phy_ram.get(x,x))


OS_ver={'10.0.0.0' : '10.0.0.0',
        '6.3.0.0' : '6.3.0.0',
        '6.1.1.0' : '6.1.1.0'}
arr=list(data.OsVer.unique())
for val in arr:
    if val not in OS_ver.keys():
        OS_ver[val]='0.0.0.0'
data.OsVer = data.OsVer.apply(lambda x: OS_ver.get(x,x))
data1.OsVer = data1.OsVer.apply(lambda x: OS_ver.get(x,x))


OS_build={17134 : 17134,
          16299 : 16299,
          15063 : 15063,
          14393 : 14393,
          10586 : 10586,
          10240 : 10240,
          9600 : 9600,
          7601 : 7601}
arr=list(data.OsBuild.unique())
for val in arr:
    if val not in OS_build.keys():
        OS_build[val]=0
data.OsBuild = data.OsBuild.apply(lambda x: OS_build.get(x,x))
data1.OsBuild = data1.OsBuild.apply(lambda x: OS_build.get(x,x))


OS_Suite={768:768,
          256:256}
arr=list(data.OsSuite.unique())
for val in arr:
    if val not in OS_Suite.keys():
        OS_Suite[val]=0
data.OsSuite = data.OsSuite.apply(lambda x: OS_Suite.get(x,x))
data1.OsSuite = data1.OsSuite.apply(lambda x: OS_Suite.get(x,x))

# ARRAYS FOR ENCODING

lab=['EngineVersion','Census_ProcessorCoreCount','Census_PrimaryDiskTotalCapacity','Census_PrimaryDiskTypeName','Census_TotalPhysicalRAM','ProductName','IsSxsPassiveMode','SmartScreen','Firewall','Census_ProcessorManufacturerIdentifier','Census_HasOpticalDiskDrive','OrganizationIdentifier','AvSigVersion','AVProductStatesIdentifier','CountryIdentifier','CityIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','OsBuild','IeVerIdentifier','Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorModelIdentifier','Census_SystemVolumeTotalCapacity','Census_InternalPrimaryDiagonalDisplaySizeInInches']
freq=['RtpStateBitfield','Platform','Processor','OsVer','OsSuite','OsPlatformSubRelease','OsBuildLab','SkuEdition','IsProtected','Census_MDC2FormFactor','Census_ChassisTypeName','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_PowerPlatformRoleName','Census_InternalBatteryNumberOfCharges']

In [ ]:
#COMBINING OF TRAIN AND TEST COLUMN
df = pd.concat([data1.assign(ind="test"), data.assign(ind="train")])

# FREQUENCY ENCODING
for i in freq:
    encoding = df.groupby(i).size()
    encoding = encoding/len(df)
    df[i] = df[i].map(encoding)

# LABEL ENCODING    
col = 'AppVersion'
def add_factor_sort(df, col):
    val = [tuple([float(v) for v in (s.split('.'))]) for s in df[col]]
    val = pd.factorize(val, sort=True)[0]
    return val
df[col] = add_factor_sort(df,col)

ordenc=OrdinalEncoder()
df[lab]=ordenc.fit_transform(df[lab])


data_test, data_train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

data_train.drop(columns=['ind'],inplace=True)
data_test.drop(columns=['ind'],inplace=True)

#NORMALIZATION OF COLUMNS

normalise=['AppVersion','EngineVersion','Census_ProcessorCoreCount','AVProductsInstalled','AVProductsEnabled','Census_PrimaryDiskTotalCapacity','Census_PrimaryDiskTypeName','Census_TotalPhysicalRAM','AvSigVersion','RtpStateBitfield','AVProductStatesIdentifier','CountryIdentifier','CityIdentifier','OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','Platform','Processor','OsVer','OsBuild','OsSuite','OsPlatformSubRelease','OsBuildLab','SkuEdition','IsProtected','IeVerIdentifier','Census_MDC2FormFactor','Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorModelIdentifier','Census_SystemVolumeTotalCapacity','Census_ChassisTypeName','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_PowerPlatformRoleName','Census_InternalBatteryNumberOfCharges']

scale=MinMaxScaler()
for col in normalise:
    data_train[[col]]=scale.fit_transform(data_train[[col]])
    data_test[[col]]=scale.transform(data_test[[col]])
    

In [ ]:
# COLUMNS managed by Ayush Sawlani

# LOADING OF DATA
df_train=pd.read_csv("../input/malware-detection-tejas/train.csv")
df_test=pd.read_csv("../input/malware-detection-tejas/test.csv")

my_train=df_train[['Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_IsSecureBootEnabled',
 'Census_IsTouchEnabled',
 'Census_IsPenCapable',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer']]


my_test=df_test[['Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing',
 'Census_FirmwareManufacturerIdentifier',
 'Census_FirmwareVersionIdentifier',
 'Census_IsSecureBootEnabled',
 'Census_IsTouchEnabled',
 'Census_IsPenCapable',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer']]


# COMBINING OF TEST AND TRAIN DATA

df = pd.concat([my_test.assign(ind="test"), my_train.assign(ind="train")])

# DATA CLEANING AND PRE-PROCESSING

df.drop(columns=['Census_OSSkuName', 'Census_OSInstallLanguageIdentifier'], inplace=True)
df=df.astype('string')
df['Census_FirmwareManufacturerIdentifier'].fillna(df['Census_FirmwareManufacturerIdentifier'].mode()[0], inplace=True)
df['Census_IsAlwaysOnAlwaysConnectedCapable'].fillna(df['Census_IsAlwaysOnAlwaysConnectedCapable'].mode()[0], inplace=True)
df['Wdft_IsGamer'].fillna(df['Wdft_IsGamer'].mode()[0], inplace=True)
df['Census_FirmwareVersionIdentifier'].fillna('OTH', inplace=True)

# GROUPING OF CATEGORIES SO THAT NUMBER OF CATEGORIES REDUCE

active_channel={'Retail' : 'Retail',
                'OEM:DM' : 'OEM',
                'Volume:GVLK' : 'Volume',
                'OEM:NONSLP' : 'OEM',
                'Volume:MAK' : 'Volume',
                'Retail:TB:Eval': 'Retail'}

df.Census_ActivationChannel = df.Census_ActivationChannel.apply(lambda x: active_channel.get(x,x))


Os_edition={'Core' : 'Core',
                'Professional' : 'Professional',
                'CoreSingleLanguage' : 'CoreSingleLanguage',
                'CoreCountrySpecific' : 'CoreCountrySpecific'}

arr=list(df.Census_OSEdition.unique())
for name in arr:
    if name not in Os_edition.keys():
        Os_edition[name]='OTH'
df.Census_OSEdition = df.Census_OSEdition.apply(lambda x: Os_edition.get(x,x))


# ENCODING

label=['Census_OSVersion', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision',  'Census_IsSecureBootEnabled', 'Census_IsTouchEnabled', 'Census_IsPenCapable', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Census_OSArchitecture', 'Census_OSInstallTypeName', 'Census_GenuineStateName', 'Census_OSWUAutoUpdateOptionsName', 'Census_OSEdition', 'Census_ActivationChannel']
frequency=['Census_OSUILocaleIdentifier','Census_FlightRing']

#LABEL ENCODING

ordenc=OrdinalEncoder()
df[label]=ordenc.fit_transform(df[label])

#FREQUENCY ENCODING

def frequency_encoding(feature):
    t = df[feature].value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[feature] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

for feature in (frequency):
    freq_enc_dict = frequency_encoding(feature)
    df[feature] = df[feature].map(lambda x: freq_enc_dict.get(x, np.nan)+1)
    df[feature] = df[feature].astype('int64')



df_test, df_train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]


df_train.drop(columns='ind', inplace=True)
df_test.drop(columns='ind', inplace=True)


#NORMALIZATION OF COLUMNS

from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
for i in df_train.columns:
    df_train[[i]]=scale.fit_transform(df_train[[i]])
    df_test[[i]]=scale.transform(df_test[[i]])



In [ ]:
# COMBINING OF OUR COLUMNS FOR FINAL TRAIN AND TEST DATA

train = pd.concat([data_train, df_train], axis=1)
test=pd.concat([data_test,df_test] , axis=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(train, has, test_size=0.2, random_state=0,stratify=has)
# This was used for testing locally

MODEL TRAINING

In [ ]:
# Model For the Original Dataset

clf=lgb.LGBMClassifier(max_depth=10,
                       n_estimators=4500,
                       learning_rate=0.006,
                       num_leaves=511,
                       min_data_in_leaf=50,
                       colsample_bytree=0.30,
                       objective='binary',
                       n_jobs=-1)

In [ ]:
clf.fit(train,has) # RUNNING MODEL 

In [ ]:
res1=clf.predict_proba(test)[:,1] # RESULT ARRAY FROM FIRST TRAINING

In [ ]:
# Random Oversampling of Data

oversample = RandomOverSampler(sampling_strategy=0.5) 
X_oversampled, y_oversampled = oversample.fit_resample(train, has)

In [ ]:
#Model For Over Sampled Data

clf=lgb.LGBMClassifier(max_depth=20,
                       n_estimators=6500,
                       learning_rate=0.002,
                       num_leaves=1023,
                       min_data_in_leaf=100,
                       colsample_bytree=0.30,
                       objective='binary',
                       n_jobs=-1)

In [ ]:
clf.fit(X_oversampled,y_oversampled) # RUNNING MODEL FOR OVERSAMPLED DATA

In [ ]:
res2=clf.predict_proba(test)[:,1] #RESULT TRAINING FROM 2ND MODEL

In [ ]:
# ENSEMBLING OF BOTH THE RESULTS

finalresult=(res1*0.75 + res2*0.25) 

In [ ]:
# CREATING RESULT DATAFRAME AND CONVERTING TO CSV FILE

result=pd.DataFrame() 
result['MachineIdentifier']=identifier['MachineIdentifier']
result['HasDetections']=finalresult
result.to_csv('final', index=False)